# Imports

In [1]:
import tensorflow as tf

import onnxruntime as ort
import numpy as np
from time import time

Run tests only on cpu

In [2]:
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "-1"

# Speed test

In [6]:
RUNS = 100

## Normal model

In [7]:
model = tf.keras.models.load_model("saved/tf_model")
input = tf.random.normal([1, 201, 498])

tik = time()
for _ in range(RUNS):
    model(input)
tok = time()
print(model(input))
print("Duration :{:.2f}ms".format(((tok-tik)/RUNS)*1000))

tf.Tensor([[-0.4653147   0.21275467]], shape=(1, 2), dtype=float32)
Duration :17.60ms


## ONNX

In [8]:
tf_random_tensor = np.random.rand(1, 201, 498)
tf_random_tensor = tf_random_tensor.astype(np.float32)

ort_sess = ort.InferenceSession('onnx/f-crnn.onnx')
outputs = ort_sess.run([], {'x': tf_random_tensor})
# Print Result
# result = outputs[0].argmax(axis=1)
print("Activations :", outputs[0][0])
# print("label :",result[0])

tik = time()
for _ in range(RUNS):
    ort_sess.run([], {'x': tf_random_tensor})
tok = time()
print("Duration :{:.2f}ms".format(((tok-tik)/RUNS)*1000))

Activations : [-1.0876914  0.2596222]
Duration :0.35ms


## TF-lite

In [11]:
%%capture
converter = tf.lite.TFLiteConverter.from_saved_model("saved/tf_model")
converter.optimizations = [tf.lite.Optimize.DEFAULT]
tflite_model = converter.convert()

Estimated count of arithmetic ops: 20.144 M  ops, equivalently 10.072 M  MACs


2022-04-12 20:19:18.352333: W tensorflow/compiler/mlir/lite/python/tf_tfl_flatbuffer_helpers.cc:357] Ignored output_format.
2022-04-12 20:19:18.352384: W tensorflow/compiler/mlir/lite/python/tf_tfl_flatbuffer_helpers.cc:360] Ignored drop_control_dependency.
2022-04-12 20:19:18.352568: I tensorflow/cc/saved_model/reader.cc:43] Reading SavedModel from: saved/tf_model
2022-04-12 20:19:18.378409: I tensorflow/cc/saved_model/reader.cc:78] Reading meta graph with tags { serve }
2022-04-12 20:19:18.378481: I tensorflow/cc/saved_model/reader.cc:119] Reading SavedModel debug info (if present) from: saved/tf_model
2022-04-12 20:19:18.429090: I tensorflow/cc/saved_model/loader.cc:228] Restoring SavedModel bundle.
2022-04-12 20:19:18.574976: I tensorflow/cc/saved_model/loader.cc:212] Running initialization op on SavedModel bundle at path: saved/tf_model
2022-04-12 20:19:18.655239: I tensorflow/cc/saved_model/loader.cc:301] SavedModel load for tags { serve }; Status: success: OK. Took 302672 micros

In [12]:
interpreter = tf.lite.Interpreter(model_content=tflite_model)
interpreter.allocate_tensors()

input_index = interpreter.get_input_details()[0]['index']
output_index = interpreter.get_output_details()[0]['index']

interpreter.set_tensor(input_index, tf.random.normal([1, 201, 498]))

tik = time()
for _ in range(RUNS):
    interpreter.invoke()
tok = time()
output = interpreter.tensor(output_index)()[0]
print(output)
print("Duration :{:.2f}ms".format(((tok-tik)/RUNS)*1000))

[-0.69725245  0.4087796 ]
Duration :23.87ms
